In [1]:
import findspark

In [2]:
# Initializing PySpark by finding the location
# findspark.init('') also works after restarting
findspark.init('/opt/anaconda3/lib/python3.11/site-packages/pyspark/')

In [3]:
from pyspark.sql import SparkSession

#Initialize SparkSession
spark = SparkSession.builder.appName("WordCount").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/25 05:55:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/09/25 05:55:38 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
from pyspark.sql.functions import *

In [5]:
test_df = [("James","Sales","NY",90000,34,10000),
    ("Michael","Sales","NY",86000,56,20000),
    ("Robert","Sales","CA",81000,30,23000),
    ("Maria","Finance","CA",90000,24,23000),
    ("Raman","Finance","CA",99000,40,24000),
    ("Scott","Finance","NY",83000,36,19000),
    ("Jen","Finance","NY",79000,53,15000),
    ("Jeff","Marketing","CA",80000,25,18000),
    ("Kumar","Marketing","NY",91000,50,21000)
  ]

In [6]:
ud_scehma = ["employee_name","department","state","salary","age","bonus"]

In [7]:
df = spark.createDataFrame(data=test_df,schema = ud_scehma)

In [8]:
df.cache().count()

9

In [9]:
from pyspark import StorageLevel

In [10]:
test = StorageLevel(useDisk=False,useMemory=True,useOffHeap=False,deserialized=False,replication=2)

In [11]:
df.persist(StorageLevel.MEMORY_AND_DISK)

23/09/25 05:55:51 WARN CacheManager: Asked to cache already cached data.


DataFrame[employee_name: string, department: string, state: string, salary: bigint, age: bigint, bonus: bigint]

In [12]:
df.unpersist()

DataFrame[employee_name: string, department: string, state: string, salary: bigint, age: bigint, bonus: bigint]

In [13]:
df.persist(storageLevel = test)

DataFrame[employee_name: string, department: string, state: string, salary: bigint, age: bigint, bonus: bigint]

In [14]:
df.groupBy('state').max("salary").alias("max_sal_by_state").show()

23/09/25 05:55:52 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
23/09/25 05:55:52 WARN BlockManager: Block rdd_18_0 replicated to only 0 peer(s) instead of 1 peers
23/09/25 05:55:53 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
23/09/25 05:55:53 WARN BlockManager: Block rdd_18_1 replicated to only 0 peer(s) instead of 1 peers


+-----+-----------+
|state|max(salary)|
+-----+-----------+
|   CA|      99000|
|   NY|      91000|
+-----+-----------+



In [15]:
df.groupBy("department","state").agg(sum("salary").alias("sum_salary"),avg("salary").alias("avg_salary")).where(col("avg_salary")>80000).show()

+----------+-----+----------+----------+
|department|state|sum_salary|avg_salary|
+----------+-----+----------+----------+
|     Sales|   CA|     81000|   81000.0|
|   Finance|   CA|    189000|   94500.0|
|     Sales|   NY|    176000|   88000.0|
|   Finance|   NY|    162000|   81000.0|
| Marketing|   NY|     91000|   91000.0|
+----------+-----+----------+----------+



In [16]:
df.createOrReplaceTempView("ida")

In [17]:
spark.sql("select * FROM ida")

DataFrame[employee_name: string, department: string, state: string, salary: bigint, age: bigint, bonus: bigint]

In [18]:
spark.sql("CREATE DATABASE idashell")

DataFrame[]

In [19]:
spark.sql("use idashell")

DataFrame[]

In [20]:
df.write.saveAsTable("test_tbl")

SparkRuntimeException: [LOCATION_ALREADY_EXISTS] Cannot name the managed table as `spark_catalog`.`idashell`.`test_tbl`, as its associated location 'file:/home/labuser/spark-warehouse/idashell.db/test_tbl' already exists. Please pick a different table name, or remove the existing location first.

In [ ]:
test = spark.sql("DESCRIBE test_tbl")

In [ ]:
test.show()

In [ ]:
test1 = spark.sql("DESCRIBE EXTENDED test_tbl")
test1.show()

In [ ]:
df.write.option("path","/home/labuser/Documents/table_data").saveAsTable("e_emptbl")

In [ ]:
test01 = spark.sql("DESCRIBE EXTENDED e_emptbl")
test01.show()

In [ ]:
spark.sql("drop table default.emp_tbl")

In [ ]:
spark.sql("drop table idashell.e_emp_tbl")